In [26]:
import numpy as np
import pandas as pd

* High Card: Highest value card.
* One Pair: Two cards of the same value.
* Two Pairs: Two different pairs.
* Three of a Kind: Three cards of the same value.
* Straight: All cards are consecutive values.
* Flush: All cards of the same suit.
* Full House: Three of a kind and a pair.
* Four of a Kind: Four cards of the same value.
* Straight Flush: All cards are consecutive values of same suit.
* Royal Flush: Ten, Jack, Queen, King, Ace, in same suit.
* The cards are valued in the order: 2, 3, 4, 5, 6, 7, 8, 9, 10, Jack, Queen, King, Ace.

There are 3 kinds of hands
* multiples : 1P, 2P, Triple, Full, Square
* sequence : Straight
* flush: Flush, StraightFlush, RoyalFlush

* If multiple, one needs to encode the value of each repeated card and the values of remaining
    * ((4,2), (5,2), (T,1)) for 2P
    * ((J,4), (3,1))
* If flush, one needs to encode Straight and Royal

In [27]:
f = open("0054_poker.txt")
l = f.readlines()
f.close()

In [86]:
hand_order = {}
hand_order["Plain"]=0
hand_order["1P"]=1
hand_order["2P"]=2
hand_order["Triple"]=3
hand_order["Straight"]=4
hand_order["Flush"]=5
hand_order["Full"]=6
hand_order["Square"]=7
hand_order["SFlush"]=8
hand_order["RFlush"]=9

def comp_hand(h1,h2, type):
    if type=="Plain":
        dh = h1.sum(axis=1).values - h2.sum(axis=1).values
        dh_abs = np.abs(dh)
        idx = np.argwhere(dh_abs>0)
        if len(idx)>0:
            res=np.sign(dh[np.max(np.argwhere(dh_abs>0))])
        else:
            res=0
    else:
        M1 = max(h1.sum(axis=1).values)
        M2 = max(h2.sum(axis=1).values)
        V1 = h1.sum(axis=1).values.argmax()
        V2 = h2.sum(axis=1).values.argmax()

        if V1>V2:
            res=1
        elif V2>V1:
            res=-1
        else:
            dh = h1.sum(axis=1).values - h2.sum(axis=1).values
            dh_abs = np.abs(dh)
            idx = np.argwhere(dh_abs>0)
            if len(idx)>0:
                res = np.sign(dh[np.max(np.argwhere(dh_abs>0))])
            else:
                res = 0
    return res

def hand(h1):
    h1_sum = h1.sum(axis=1).to_frame().rename(columns={0:"cnt"})
    M = max(h1_sum["cnt"])
        
    if M==2 and h1_sum.loc[h1_sum.cnt == M].shape[0]==1:
        hand_type = "1P"
    
    if M==2 and h1_sum.loc[h1_sum.cnt == M].shape[0]==2:
        hand_type = "2P"
    
    if M==3 and h1_sum.loc[h1_sum.cnt == 2].shape[0]==0:
        hand_type = "Triple"
    
    if M==3 and h1_sum.loc[h1_sum.cnt == 2].shape[0]==1:
        hand_type = "Full"
    
    if M==4:
        hand_type = "Square"
    
    if M==1:
        h1_vsum = h1.sum(axis=0).to_frame().rename(columns={0:"cnt"})
        if max(h1_vsum["cnt"]) <5:
            h1_hsum = h1.sum(axis=1).to_frame().rename(columns={0:"cnt"})
            h1_hsum.reset_index(inplace=True)
            idx=[int(i) for i in h1_hsum.loc[h1_hsum.cnt==1].index]
            if max(idx) - min(idx) == 4:
                hand_type="Straight"
            else:
                hand_type = "Plain"
        else:
            cname = h1_vsum.loc[h1_vsum.cnt==5, "cnt"].index[0]
            h1_tmp = h1.reset_index(drop=True)
            idx = [int(i)  for i in list(h1_tmp.loc[h1_tmp[cname]==1, cname].index)]
            if max(idx) - min(idx) == 4:
                if h1.loc["A",cname]==1:
                    hand_type = "RFlush"
                else:
                    hand_type = "SFlush"
            else:
                hand_type = "Flush"
    return hand_type

In [90]:
%%time

score1=0
score2=0
even=0
even_idx=[]
df_list = []
for i in np.arange(len(l)):

    h1 = pd.DataFrame(np.zeros((13,4), dtype='int'), 
                 index=[str(i) for i in list(np.arange(2,10))] + ['T','J','Q','K','A'], 
                 columns=['H','D','C','S'])
    for c in l[i].strip().split(' ')[:5]:
        h1.loc[c[0],c[1]]=1
    
    h2 = pd.DataFrame(np.zeros((13,4), dtype='int'), 
                 index=[str(i) for i in list(np.arange(2,10))] + ['T','J','Q','K','A'], 
                 columns=['H','D','C','S'])        
    for c in l[i].strip().split(' ')[5:]:
        h2.loc[c[0],c[1]]=1

    hand_h1 = hand(h1)
    hand_h2 = hand(h2)
    
    if(hand_order[hand_h1]>hand_order[hand_h2]):
        score1+=1
        df_list.append(pd.DataFrame({"idx":[i],"h1":1,"h2":0}))
    if(hand_order[hand_h1]<hand_order[hand_h2]):
        score2+=1
        df_list.append(pd.DataFrame({"idx":[i],"h1":0,"h2":1}))
        
    if(hand_order[hand_h1]==hand_order[hand_h2]) and hand_h1 in ["Plain", "1P"]:
        if comp_hand(h1,h2,hand_h1)>=1:
            score1+=1
            df_list.append(pd.DataFrame({"idx":[i],"h1":1,"h2":0}))
        elif comp_hand(h1,h2,hand_h1)<=-1:
            score2+=1
            df_list.append(pd.DataFrame({"idx":[i],"h1":0,"h2":1}))
        else:
            even+=1
            even_idx+=[i]

score1

CPU times: user 3.48 s, sys: 135 ms, total: 3.62 s
Wall time: 3.52 s


376